## Plano do Projeto - Classificação de Dígitos MNIST com Redes Neurais e Cálculo de Métricas de Avaliação de Aprendizado.

1. Definição do problema: Criar uma rede neural para classifcar imagens de dígitos manuscritos (0 a 9) do dataset MNIST e avaliar o desempenho do modelo com sensibilidade, especificidade, precisão, acurácia e F-score.

2. Pré -processamento dos dados: 
* carregar o dataset MNIST;
* Normalizar os valores dos pixels (escala 0 a 1);
* Converter os rótulos em one-hot encoding (para usar na classificação com Softmax);
* Dividir os dados em treino (80%) e (20%) teste.

3. Construção da rede Neural:
* Entrada: 28x28 pixels (convertidos em um vetor de 784 elementos);
* Camadas ocultas: 2 camadas densas (fully connected) com ativação ReLU;
* Saída: 10 neurônios (um para cada dígito) com ativação Sofmax;
* Função perda: Categorical Crossentropy;
* Otimizador: Adam.

4. Treinamento do Modelo:
* Utilizar batch size de 32 e treinar por 10 épocas;
* Acompanhar acurácia e perda no conjunto de validação.

5. Avaliação com as Métricas do Desafio:
* Fazer previsões no conjunto de teste;
* Construir a matriz de confusão (comparando rótulos reais e previstos);
* Calcular sensibilidade, especificidade, precisão, acurácia e F-score para cada classe.




----------------------------------

1. Importação das bibliotecas necessárias:

In [18]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.datasets import mnist 

from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

2. * Carregar e Pré-processar os dados;
   * Normalização (deixar os valores dos pixels entre 0 e 1);
   * One-hot encoding para os módulos;

In [19]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalização e reformatação
x_train = x_train.reshape(-1, 28*28) / 255.0
x_test = x_test.reshape(-1,28*28) / 255.0

# One-hot encoding para os rótulos
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test,10) # Guardamos y_test original para a matriz de confusão


3. Criar a Rede Neural

In [20]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(784,)),  # Define a entrada corretamente
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compilar o Modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

4. Treinar o Modelo e fazer previsões:

In [21]:
model.fit(x_train, y_train_cat, epochs=10, batch_size=32, validation_data=(x_test, y_test_cat))

#previsões
y_pred_prob = model.predict(x_test)
y_pred = np.argmax(y_pred_prob, axis=1) # Converte probabilidades em rótulos inteiros

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8750 - loss: 0.4285 - val_accuracy: 0.9617 - val_loss: 0.1245
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9695 - loss: 0.1030 - val_accuracy: 0.9737 - val_loss: 0.0865
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9784 - loss: 0.0685 - val_accuracy: 0.9739 - val_loss: 0.0868
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9829 - loss: 0.0518 - val_accuracy: 0.9762 - val_loss: 0.0847
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9878 - loss: 0.0378 - val_accuracy: 0.9736 - val_loss: 0.0847
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9895 - loss: 0.0322 - val_accuracy: 0.9779 - val_loss: 0.0806
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9918 - loss: 0.0246 - val_accuracy: 0.9784 - val_loss: 0.0760
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9925 - loss: 0.0207 - 

5. Calculo da matriz confusão:

In [22]:
cm = confusion_matrix(y_test, y_pred)

6. Cálculo das métricas para cada classe (0 a 9)

In [23]:
def calcular_metricas(cm):
    metricas = {}
    for i in range(10):  # Para cada classe (0 a 9)
        VP = cm[i, i]
        FN = cm[i, :].sum() - VP
        FP = cm[:, i].sum() - VP
        VN = cm.sum() - (VP + FP + FN)

        sensibilidade = VP / (VP + FN) if (VP + FN) > 0 else 0
        especificidade = VN / (VN + FP) if (VN + FP) > 0 else 0
        acurácia = (VP + VN) / cm.sum() if cm.sum() > 0 else 0
        precisao = VP / (VP + FP) if (VP + FP) > 0 else 0
        f_score = 2 * (precisao * sensibilidade) / (precisao + sensibilidade) if (precisao + sensibilidade) > 0 else 0

        metricas[f'Dígito {i}'] = [sensibilidade, especificidade, acurácia, precisao, f_score]
    
    return metricas

7. Resultados

In [24]:
metricas_resultado = calcular_metricas(cm)
df_resultado = pd.DataFrame.from_dict(metricas_resultado, orient='index',
                                      columns=["Sensibilidade", "Especificidade", "Acurácia", "Precisão", "F-score"])

print(df_resultado)

          Sensibilidade  Especificidade  Acurácia  Precisão   F-score
Dígito 0       0.985714        0.998448    0.9972  0.985714  0.985714
Dígito 1       0.987665        0.999098    0.9978  0.992914  0.990283
Dígito 2       0.977713        0.997324    0.9953  0.976767  0.977240
Dígito 3       0.976238        0.997998    0.9958  0.982072  0.979146
Dígito 4       0.982688        0.995564    0.9943  0.960199  0.971314
Dígito 5       0.973094        0.998682    0.9964  0.986364  0.979684
Dígito 6       0.967641        0.999005    0.9960  0.990385  0.978881
Dígito 7       0.987354        0.997548    0.9965  0.978785  0.983051
Dígito 8       0.974333        0.997119    0.9949  0.973333  0.973833
Dígito 9       0.980178        0.996441    0.9948  0.968658  0.974384
